In [61]:
# !pip install openai==0.28

In [ ]:
import openai

# see: https://platform.openai.com/docs/quickstart
openai.api_key = ""  # add your key here
from IPython.display import display, Markdown
import requests
import json

In [63]:
# Define the DotDict class
class DotDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __init__(self, dct):
        for key, value in dct.items():
            if hasattr(value, "keys"):
                value = DotDict(value)
            self[key] = value

In [64]:
def clinical_trials_api(condition, terms, intervention, topK=5):
    assert topK <= 10

    # Define the payload for the request
    payload = {
        "query.cond": condition,
        "query.term": terms,
        "query.intr": intervention,
        # "filter.overallStatus": 'COMPLETED',
    }

    # Define the URL and make the request
    url = "https://clinicaltrials.gov/api/v2/studies"
    response = requests.get(url, params=payload)

    trial_ids = []
    clinical_trial_data = []

    # Check if the request was successful
    if response.status_code == 200:
        try:
            data = response.json()
            studies = data.get("studies", [])[:topK]

            for idx, study_data in enumerate(studies):
                study = DotDict(study_data)
                try:
                    # Get the study number (nctId)
                    nct_id = study.protocolSection.identificationModule.nctId
                    trial_ids.append("https://clinicaltrials.gov/study/" + str(nct_id))
                    clinical_trial_data.append(
                        "Clinical Trial Details #{} in Nested JSON format:".format(
                            idx + 1
                        )
                    )
                    clinical_trial_data.append(json.dumps(study))
                except (KeyError, AttributeError):
                    continue

            # Save clinical trial data as a string in results_dict
            clinical_trial_data = "\n".join(clinical_trial_data)

        except requests.exceptions.JSONDecodeError:
            print("Failed to decode JSON. Here is the raw response:")
            print(response.text)
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)

    # Print the collected clinical trial data
    print("The following top {} clinical trials are pulled:".format(topK))
    print("\n".join(trial_ids))
    # print(clinical_trial_data)

    return trial_ids, clinical_trial_data

In [65]:
def chatgpt_summary(context):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # "o1-mini" (more expensive) or "gpt-4o-mini" (cheaper)
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Please summarize the most clinically relevant information out of these nested JSON responses from a Clinical Trial Database for various related clinical trials. Please start with summary right away without any small talk. This is the trial data: {}".format(
                    context
                ),
            },
        ],
    )
    display(Markdown(response["choices"][0]["message"]["content"]))
    return response

In [66]:
def chatgpt_ask(context, question):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # "o1-mini" (more expensive) or "gpt-4o-mini" (cheaper)
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Please thoroughly understand clinically important information encoded in the following nested JSON responses from a Clinical Trial Database for various related clinical trials. Please answer the subsequent questions based on this data. Please be factual and always cite the clinical trial source when you use a certain piece of information from there. Here is the trial data: {}".format(
                    context
                ),
            },
            {
                "role": "user",
                "content": "Now answer the following question: {}".format(question),
            },
        ],
    )
    display(Markdown(response["choices"][0]["message"]["content"]))
    return response

# Demo

In [67]:
condition = "infarction"
terms = "diabetes"
intervention = "heart attack"
top_number_of_trials_to_consider = 3

question = """

What are the innovative ideas have been introduced in the clinical trials recently that are quite differentiating from the regular/conventional way 
of executing clinical trials? Do the trials look hopeful at all? Are any of them still recruiting?

"""

In [68]:
clinical_trials_context = clinical_trials_api(
    condition, terms, intervention, topK=top_number_of_trials_to_consider
)

The following top 3 clinical trials are pulled:
https://clinicaltrials.gov/study/NCT05332262
https://clinicaltrials.gov/study/NCT06245980
https://clinicaltrials.gov/study/NCT02542774


In [69]:
response = chatgpt_summary(clinical_trials_context)

### Summary of Clinical Trials

1. **Trial NCT05332262: ABCD-Gene Prospective Study**
   - **Objective**: Validate the ABCD-GENE score for identifying stable coronary artery disease (CAD) patients at risk of peri-procedural myocardial infarction during elective percutaneous coronary intervention (PCI).
   - **Participants**: 500 stable CAD patients undergoing elective PCI, treated with clopidogrel.
   - **Institution**: University of Florida.
   - **Status**: Recruiting, with an estimated completion date of August 2025.
   - **Primary Outcome**: Rate of peri-PCI myocardial infarction or myocardial injury within 24 hours, comparing high (≥10) vs. low (<10) ABCD-GENE score groups.
   - **Key Details**: Patients will be assessed for high platelet reactivity (HPR) post-PCI and may benefit from tailored antiplatelet therapy.

2. **Trial NCT06245980: Role of SGLT2I in Diabetic Patients With Myocardial Infarction**
   - **Objective**: Determine the effect of SGLT2 inhibitors (SGLT2i) on diabetic patients with acute myocardial infarction.
   - **Participants**: Estimated 1200 diabetic patients with acute myocardial infarction.
   - **Institution**: Chongqing Medical University, China.
   - **Status**: Recruiting, with results anticipated by March 2024.
   - **Primary Outcomes**: Major adverse cardiovascular events (MACE) including cardiovascular death, heart failure, and recurrent myocardial infarction over 24 months.
   - **Eligibility**: Inclusion criteria focus on patients with type 2 diabetes suffering from an acute myocardial infarction.

3. **Trial NCT02542774: Sex Differences in Cardiovascular Risk Associated With Diabetes**
   - **Objective**: Investigate sex differences in cardiovascular disease risks among diabetes patients, focusing on healthcare delivery effectiveness.
   - **Participants**: Estimated 80,000 diabetes patients, analyzed via existing electronic health records.
   - **Institution**: University College, London.
   - **Status**: Active but not recruiting (as of the last update in 2015).
   - **Primary Outcome**: First occurrence of any cardiovascular disease within 10 years, including myocardial infarction and heart failure.
   - **Relevance**: Findings may provide insights into gender disparities in diabetes management and cardiovascular outcomes.

These trials collectively focus on enhancing risk assessment and treatment efficacy in patients with cardiovascular conditions, particularly emphasizing the integration of genetic and therapeutic strategies to improve outcomes in diverse patient populations.

In [70]:
response = chatgpt_ask(clinical_trials_context, question.strip())

The three clinical trials presented show innovative approaches that differentiate them from conventional practices in several ways:

1. **NCT05332262 (University of Florida)**:
   - **Precision Medicine Approach**: This trial utilizes a precision medicine tool—the ABCD-GENE score—that integrates genetic data with clinical variables to identify patients at high risk for peri-procedural myocardial infarction or myocardial injury during elective percutaneous coronary intervention (PCI). This contrasts with conventional methods that generally do not incorporate genetic profiling in risk assessment.
   - **Cohort Stratification**: Patients are stratified into high and low-risk cohorts based on their ABCD-GENE score, allowing tailored interventions based on individual risk profiles.
   - **Status**: This trial is currently **recruiting** participants, with an estimated enrollment of 500 stable coronary artery disease patients (NIH, 2024).

2. **NCT06245980 (Chongqing Medical University)**:
   - **Focus on SGLT2 Inhibitors**: The trial specifically examines the role of sodium-glucose cotransporter 2 inhibitors (SGLT2i) in diabetic patients who have suffered a myocardial infarction. It aims to understand the impact of this newer class of diabetic medications on major adverse cardiovascular events (MACE).
   - **Patient Inclusion**: The trial focuses on a specific patient subset (diabetic patients with recent myocardial infarction), which allows for targeted interventions that are currently not standard in regular clinical care.
   - **Status**: This trial is also currently **recruiting**, with an estimated enrollment of 1200 participants (NIH, 2024).

3. **NCT02542774 (University College, London)**:
   - **Long-Term and Large-Scale Data Analysis**: This trial employs a retrospective cohort design based on linked electronic health records rather than standard prospective monitoring. It seeks to evaluate the cardiovascular risks associated with diabetes management, providing insights into real-world clinical practice and outcomes.
   - **Innovative Use of Data**: By analyzing existing health databases, it captures a larger population (estimated 80,000 participants) and generates findings that may significantly impact health service delivery and equality in diabetes care.
   - **Status**: Although this trial was marked as active, its recruitment status is currently **unknown** (NIH, 2024).

### Conclusion on Hopefulness and Recruitment Status
The first two trials (NCT05332262 and NCT06245980) appear hopeful and are actively recruiting participants, indicating ongoing commitment to advancing clinical practices in cardiology and diabetes. They incorporate innovative methodologies, such as genetic profiling and novel drug interventions, which are likely to offer more precise and effective treatment strategies compared to conventional methods. The third trial, however, has an uncertain recruitment status, but its large-scale data analysis is vital for understanding broader trends and outcomes in patient care.

### Sources:
- ClinicalTrials.gov for NCT05332262 (University of Florida, 2024).
- ClinicalTrials.gov for NCT06245980 (Chongqing Medical University, 2024).
- ClinicalTrials.gov for NCT02542774 (University College, London, 2024).